In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [2]:
test_message = "Rotator Night Observing integration test"

The next cell opens the database for query. 

In [3]:
con = sqlite3.connect(os.path.expanduser("~/develop/one_filt_v1.4_0yrs_1.db"))

Once the database is opened/connected we can query for all the entries in the `SummaryAllProps` database, which contains the information about each target observed. 

In [4]:
df = pd.read_sql_query("SELECT * from SummaryAllProps", con)

In [5]:
df

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,numExposures,airmass,...,rotTelPos,moonAz,sunAz,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase
0,0,295.684859,-13.859434,59853.984387,0.0,30.0,r,158.610110,1,1.042546,...,25.191637,269.661072,258.601711,3.277185,-0.058527,263.916158,-26.676875,32.316524,106.507629,42.290643
1,1,297.624917,-9.265477,59853.984808,0.0,30.0,r,161.439781,1,1.072366,...,25.832375,269.607070,258.520229,3.277192,-0.058530,263.920628,-26.677016,36.298390,108.979377,42.292634
2,2,300.107570,-7.390781,59853.985222,0.0,30.0,r,165.861747,1,1.089626,...,25.873543,269.553836,258.439914,3.277198,-0.058533,263.925032,-26.677152,39.377626,111.637958,42.294595
3,3,303.174590,-8.168721,59853.985640,0.0,30.0,r,172.377374,1,1.089597,...,25.884500,269.499995,258.358690,3.277205,-0.058536,263.929481,-26.677288,41.515365,114.558039,42.296577
4,4,305.627117,-6.189116,59853.986054,0.0,30.0,r,175.273585,1,1.112916,...,25.921403,269.446620,258.278176,3.277211,-0.058539,263.933888,-26.677420,44.623175,117.221760,42.298539
5,5,302.572304,-5.442858,59853.986471,0.0,30.0,r,168.889451,1,1.110316,...,25.902881,269.392755,258.196929,3.277218,-0.058542,263.938332,-26.677550,42.486721,114.287797,42.300519
6,6,305.026027,-3.470824,59853.986884,0.0,30.0,r,171.916349,1,1.135718,...,25.939527,269.339251,258.116231,3.277225,-0.058544,263.942742,-26.677678,45.616256,116.922472,42.302483
7,7,304.436737,-0.762268,59853.987297,0.0,30.0,r,169.313566,1,1.162791,...,25.950925,269.285737,258.035524,3.277231,-0.058547,263.947149,-26.677802,46.726982,116.563523,42.304446
8,8,303.855479,1.937510,59853.987710,0.0,30.0,r,167.094399,1,1.194027,...,25.961423,269.232110,257.954649,3.277238,-0.058550,263.951561,-26.677925,47.941773,116.145752,42.306411
9,9,301.388735,-0.024061,59853.988123,0.0,30.0,r,163.565452,1,1.162831,...,25.945989,269.178425,257.873691,3.277244,-0.058553,263.955975,-26.678045,44.787693,113.572018,42.308377


In [6]:
d = salobj.Domain()

In [7]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [8]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 10 messages


[None, None, None]

RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 11 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 12 messages


In [9]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteEvent(Rotator, 0, heartbeat) falling behind; read 17 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 79 messages


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

Start the loop on the different field positions df given by the scheduler

In [10]:
### Read the scheduler output positions for the next 8 hours
print("Number of visits:", len(df))

Number of visits: 971


In [11]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
print("This is the start of the observing run")
## start the loop that take a sky position one at the time and point and track

for field in range(len(df)): 
#for field in range(50): 
    print(field)
    target_name = df["note"][field]
    alt = df['altitude'][field] * u.deg
    az = df['azimuth'][field] * u.deg
    rot_tel = Angle(df["rotTelPos"][field]*u.deg)
    para_ang = Angle(df["paraAngle"][field]*u.deg)
    rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
    visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
    print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
    time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
    cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

    print(f"RA: {cmd_radec.ra}\nDec: {cmd_radec.dec}\nRotPA: {rot_pa}")

    if field == 0:
        await rot.cmd_trackStart.start(timeout=30.)
    
    ack = await mtptg.cmd_raDecTarget.set_start(
       targetName=target_name,
       targetInstance=ATPtg.TargetInstances.CURRENT,
       frame=ATPtg.CoordFrame.ICRS,
        epoch=2000,  # should be ignored: no parallax or proper motion
        equinox=2000,  # should be ignored for ICRS
        ra=cmd_radec.ra.hour,
        declination=cmd_radec.dec.deg,
        parallax=0,
        pmRA=0,
        pmDec=0,
        rv=0,
        dRA=0,
        dDec=0,
        rotPA=rot_pa.deg,
        rotFrame=ATPtg.RotFrame.TARGET,
        rotMode=ATPtg.RotMode.FIELD,
        timeout=10.
    )

    print("Waiting", visit_time, "s")
    await asyncio.sleep(visit_time)
    
    #we need to stop the pointing commands before stopping the rotator. This will be integrated in the pointing code
await mtptg.cmd_stopTracking.start(timeout=30)
await rot.cmd_stop.start(timeout=30)


await asyncio.sleep(5.)
script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"{test_message}:[END]")


RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages


This is the start of the observing run
0
Alt: 73.57495921073964 deg
Az: 4.273461703999071 deg
RotTel: 25.191637478972904 deg


RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 100 messages


RA: 275.53279454171985 deg
Dec: -13.871104443976728 deg
RotPA: 28.993384547298348 deg


RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 100 messages


Waiting 31.0 s
1
Alt: 68.83074605748851 deg
Az: 8.315007012391357 deg
RotTel: 25.832374501812186 deg
RA: 277.54722024269375 deg
Dec: -9.281081511954035 deg
RotPA: 33.10452967576464 deg
Waiting 31.0 s
2
Alt: 66.59882514695401 deg
Az: 13.502094205548591 deg
RotTel: 25.87354311047927 deg
RA: 280.01267806326007 deg
Dec: -7.41105588821426 deg
RotPA: 37.60883293952574 deg
Waiting 31.0 s
3
Alt: 66.60224475932411 deg
Az: 21.042108021538045 deg
RotTel: 25.884499929472675 deg
RA: 283.0574920423376 deg
Dec: -8.194588747766817 deg
RotPA: 44.14637372865934 deg
Waiting 31.0 s
4
Alt: 63.9668480605217 deg
Az: 24.58666100047474 deg
RotTel: 25.92140309499105 deg
RA: 285.49295277914337 deg
Dec: -6.21953184364724 deg
RotPA: 47.116391227376795 deg
Waiting 31.0 s
5
Alt: 64.24286580031655 deg
Az: 17.11028898548473 deg
RotTel: 25.90288127205781 deg
RA: 282.4195938045749 deg
Dec: -5.467677132518491 deg
RotPA: 40.69521353385676 deg
Waiting 31.0 s
6
Alt: 61.70275581044189 deg
Az: 20.749621108102865 deg
RotTel: 2

In [26]:
await rot.cmd_clearError.start(timeout=10)

RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 100 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 77 messages


In [ ]:
await asyncio.sleep(5)
await salobj.set_summary_state(mtptg, salobj.State.STANDBY)
await salobj.set_summary_state(rot, salobj.State.STANDBY)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 20 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 11 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 21 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 21 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 11 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 11 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 11 messages
